# Projeto 2 - Ciência dos Dados

Nome: Beatriz Muniz de Castro e Silva

Nome: Nicole Sarvasi Alves da Costa

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [6]:
%%capture

#Instalando o tweepy
!pip install tweepy
#Instalando o pacote emoji para limpar mensagens
!pip install emoji

In [7]:
#importando os pacotes necessários
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
import functools
import operator
import numpy as np
import emoji
import re

___
## Autenticando no  Twitter

* Conta: @nicknennis

In [8]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @nickcnennis

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [9]:
#Produto escolhido:
produto = 'bacurau'

#Quantidade mínima de mensagens capturadas:
n = 700
#Quantidade mínima de mensagens para a base de treinamento:
t = 500

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [10]:
#Cria um objeto para a captura
api = tweepy.API(auth, wait_on_rate_limit=True)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []

#retira os retweets
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode='extended').items():  
    if (not msg.retweeted) and ('RT' not in msg.full_text): 
        msgs.append(msg.full_text.lower())
        i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

TweepError: Twitter error response: status code = 401

Salvando os dados em uma planilha Excel:

In [15]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

In [38]:
#leitura do arquivo excel
treino = pd.read_excel("bacurau.xlsx", "Treinamento") #tabela da parte de treinamento do excel
teste = pd.read_excel("bacurau.xlsx", "Teste") #tabela da parte de teste do excel

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [12]:
def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    import string
    
    text = ' '.join(word for word in text.split() if not word.startswith('https'))
    
    text_split_emoji = emoji.get_emoji_regexp().split(text)
    text_split_whitespace = [substr.split() for substr in text_split_emoji]
    text_split = functools.reduce(operator.concat, text_split_whitespace)
    text = ' '.join(word for word in text_split)
    
    punctuation = '[!-/.:?;@]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    nova_linha = '[\n]'
    text_subbed = re.sub(pattern, ' ', text)
    text_subbed = re.sub(nova_linha, " ", text_subbed)
    return text_subbed

In [95]:
#criar uma lista com os tweets de teste limpos (sem @ e essas negocias loucas)
teste_t = pd.Series(teste["Teste"])
teste_limpo = []
for i in teste_t:
    teste_limpo.append(cleanup(i.lower()))
# teste_limpo

In [97]:
#criar uma lista com os tweets de treinamento limpos (same as above)
treino_s = pd.Series(treino["Treinamento"])
treino_limpo = []
for i in treino_s:
    treino_limpo.append(cleanup(str(i).lower()))
# treino_limpo

In [86]:
treino_t = (" ").join(treino_limpo) #junta todos os tweets de treinamento em uma string só
lista_relativa = treino_t.split() #divide a string pra fazer uma lista com cada palavra separadamente
frequencia_absoluta = pd.Series(lista_relativa).value_counts()
frequencia_relativa = pd.Series(lista_relativa).value_counts(True)
frequencia_relativa #frequencia relativa total das palavras

bacurau       0.059806
de            0.044272
e             0.021100
rt            0.021100
a             0.019288
é             0.017864
que           0.017735
tirei         0.015534
o             0.015146
eu            0.012816
você          0.011133
do            0.009968
ver           0.009191
pra           0.008932
no            0.008026
um            0.007896
assistir      0.007896
filme         0.007249
não           0.007120
uma           0.006861
com           0.006731
para          0.006731
em            0.006084
da            0.005955
tem           0.005696
mas           0.005437
gente         0.005437
qual          0.004919
domingas      0.004531
lunga         0.004531
                ...   
tirar         0.000129
esta          0.000129
aquela        0.000129
pensada       0.000129
cadê          0.000129
molena        0.000129
vivi          0.000129
buteco        0.000129
outro         0.000129
cmg           0.000129
ninja         0.000129
gravações     0.000129
arrecada   

In [34]:
rel = treino.index[treino.Relevância == 1] #critério para selecionar só os tweets relevantes
treino_r = treino.loc[rel, "Treinamento"]  #cria nova database com os tweets relevantes


#frequência das palavras nos tweets relevantes
treino_rt = treino_r.str.cat()
treino_rt = cleanup(treino_rt.lower())
lista_relevante = treino_rt.split()
frequencia_absoluta = pd.Series(lista_relevante).value_counts()
frequencia_rel_relevantes = pd.Series(lista_relevante).value_counts(True)
frequencia_rel_relevantes #frequência relativa dos tweets relevantes

bacurau            0.059778
de                 0.034291
que                0.027340
e                  0.024096
o                  0.018999
pra                0.018536
ver                0.018072
a                  0.016682
assistir           0.014365
filme              0.013438
é                  0.012975
do                 0.012975
eu                 0.012512
no                 0.010658
não                0.009268
um                 0.007414
em                 0.007414
mas                0.006951
cinema             0.006951
com                0.006951
mais               0.006487
me                 0.006487
tem                0.005561
uma                0.005097
sobre              0.005097
bom                0.005097
vou                0.005097
da                 0.004634
vai                0.004634
q                  0.004634
                     ...   
existem            0.000463
centro             0.000463
sido               0.000463
paga               0.000463
novoalguém         0

In [35]:
irel = treino.index[treino.Relevância == 0]
treino_ir = treino.loc[irel, "Treinamento"]  #tweets irrelevantes

#frequência das palavras nos tweets irrelevantes
treino_irt = treino_ir.str.cat()
treino_irt = cleanup(treino_irt.lower())
lista_irrelevante = treino_irt.split()
frequencia_absoluta = pd.Series(lista_irrelevante).value_counts()
frequencia_rel_irrelevantes = pd.Series(lista_irrelevante).value_counts(True)
frequencia_rel_irrelevantes #frequência relativa dos tweets irrelevantes

bacurau            0.054954
de                 0.051858
é                  0.021091
a                  0.020511
e                  0.020511
que                0.014899
tirei              0.014319
você               0.014319
o                  0.014125
eu                 0.011997
do                 0.009481
para               0.009094
um                 0.008707
uma                0.007933
no                 0.007546
com                0.007159
da                 0.006966
domingas           0.006772
qual               0.006772
lunga              0.006579
não                0.006385
gente              0.006192
em                 0.005998
personagem         0.005998
tem                0.005998
assistir           0.005805
ver                0.005805
pra                0.005611
se                 0.005418
filme              0.005224
                     ...   
coragem            0.000193
rondelicia         0.000193
tropa              0.000193
cineart            0.000193
água               0

In [120]:
quantidade_relevancia = treino.Relevância.value_counts()
quantidade_relevancia #quantidade de tweets relevantes e irrelevantes

0    354
1    147
Name: Relevância, dtype: int64

In [119]:
quantidade_relevancia_pct = treino.Relevância.value_counts(True)*100
quantidade_relevancia_pct #porcentagem dos tweets que são relevantes e irrelevantes

0    70.658683
1    29.341317
Name: Relevância, dtype: float64

In [109]:
palavras_totais = [] #lista com todas as palavras sem duplicatas

for palavra in lista_relativa:
    if palavra not in palavras_totais:
        palavras_totais.append(palavra)
for palavra in lista_relevante:
    if palavra not in palavras_totais:
        palavras_totais.append(palavra)
for palavra in lista_irrelevante:
    if palavra not in palavras_totais:
        palavras_totais.append(palavra)    
print(palavras_totais)
    

['nossa', 'eu', 'desviei', 'muito', 'rápido', 'de', 'um', 'spoiler', 'bacurau', 'me', 'senti', 'ninja', 'rt', 'avokdoido', 'quem', 'nao', 'gostou', 'é', 'no', 'mínimo', 'do', 'pouco', 'nazista', 'sim', 'vi', 'ontem', 'com', 'a', 'viviane_cardoso', 'o', 'filme', 'fantástico', 'vale', 'pena', 'ver', 'se', 'você', 'tiver', 'mente', 'aberta', 'e', 'qualquer', 'coisa', 'que', 'faço', 'vivi', 'sempre', 'bom', 'companheira', 'filmes', 'vida', 'lutas', 'lt', '3', 'hoje', 'finalmente', 'vejo', 'será', 'consigo', 'chegar', 'até', 'sala', 'livre', 'tá', 'difícil', 'lzanin', 'texto', 'marcelo', 'coelho', 'chama', '‘bolsonarisno', 'sinal', 'trocado’', 'lembra', 'ideia', 'alguns', 'jornalistas', 'na', 'é…', 'diretor', 'joão', 'kleber', 'mendonça', 'filho', 'n', 'vou', 'conseguir', 'qual', 'desse', 'q', 'galera', 'ta', 'doida', 'achei', 'era', 'peixe', 'isso', 'vai', 'assistir', 'sai', 'maluco', 'todo', 'dia', 'bora', 'galeres', 'aproveitando', 'pra', 'divulgar', 'melhor', '2019', 'personagem', 'tire

In [114]:
freq_irr={}
freq_rel={}

for palavra in palavras_totais:
    freq_irr[palavra]=1
    freq_rel[palavra]=1x

{'nossa': 3, 'eu': 28, 'desviei': 1, 'muito': 6, 'rápido': 1, 'de': 75, 'um': 17, 'spoiler': 2, 'bacurau': 130, 'me': 15, 'senti': 1, 'ninja': 1, 'rt': 6, 'avokdoido': 2, 'quem': 8, 'nao': 5, 'gostou': 2, 'é': 29, 'no': 24, 'mínimo': 3, 'do': 29, 'pouco': 2, 'nazista': 2, 'sim': 5, 'vi': 5, 'ontem': 6, 'com': 16, 'a': 37, 'viviane_cardoso': 2, 'o': 42, 'filme': 30, 'fantástico': 2, 'vale': 2, 'pena': 3, 'ver': 40, 'se': 6, 'você': 7, 'tiver': 2, 'mente': 2, 'aberta': 2, 'e': 53, 'qualquer': 3, 'coisa': 8, 'que': 60, 'faço': 2, 'vivi': 2, 'sempre': 3, 'bom': 12, 'companheira': 2, 'filmes': 7, 'vida': 4, 'lutas': 2, 'lt': 2, '3': 1, 'hoje': 8, 'finalmente': 3, 'vejo': 2, 'será': 2, 'consigo': 2, 'chegar': 2, 'até': 4, 'sala': 3, 'livre': 2, 'tá': 9, 'difícil': 2, 'lzanin': 2, 'texto': 2, 'marcelo': 2, 'coelho': 2, 'chama': 2, '‘bolsonarisno': 2, 'sinal': 2, 'trocado’': 2, 'lembra': 2, 'ideia': 2, 'alguns': 2, 'jornalistas': 2, 'na': 8, 'é…': 1, 'diretor': 1, 'joão': 1, 'kleber': 1, 'mend

In [115]:
for palavra in lista_relevante:
    freq_rel[palavra]+=1
freq_rel

{'nossa': 5,
 'eu': 55,
 'desviei': 1,
 'muito': 11,
 'rápido': 1,
 'de': 149,
 'um': 33,
 'spoiler': 3,
 'bacurau': 259,
 'me': 29,
 'senti': 1,
 'ninja': 1,
 'rt': 11,
 'avokdoido': 3,
 'quem': 15,
 'nao': 9,
 'gostou': 3,
 'é': 57,
 'no': 47,
 'mínimo': 5,
 'do': 57,
 'pouco': 3,
 'nazista': 3,
 'sim': 9,
 'vi': 9,
 'ontem': 11,
 'com': 31,
 'a': 73,
 'viviane_cardoso': 3,
 'o': 83,
 'filme': 59,
 'fantástico': 3,
 'vale': 3,
 'pena': 5,
 'ver': 79,
 'se': 11,
 'você': 13,
 'tiver': 3,
 'mente': 3,
 'aberta': 3,
 'e': 105,
 'qualquer': 5,
 'coisa': 15,
 'que': 119,
 'faço': 3,
 'vivi': 3,
 'sempre': 5,
 'bom': 23,
 'companheira': 3,
 'filmes': 13,
 'vida': 7,
 'lutas': 3,
 'lt': 3,
 '3': 1,
 'hoje': 15,
 'finalmente': 5,
 'vejo': 3,
 'será': 3,
 'consigo': 3,
 'chegar': 3,
 'até': 7,
 'sala': 5,
 'livre': 3,
 'tá': 17,
 'difícil': 3,
 'lzanin': 3,
 'texto': 3,
 'marcelo': 3,
 'coelho': 3,
 'chama': 3,
 '‘bolsonarisno': 3,
 'sinal': 3,
 'trocado’': 3,
 'lembra': 3,
 'ideia': 3,
 'alg

In [116]:
for palavra in lista_irrelevante:
    freq_irr[palavra]+=1
freq_irr

{'nossa': 9,
 'eu': 125,
 'desviei': 3,
 'muito': 33,
 'rápido': 3,
 'de': 537,
 'um': 91,
 'spoiler': 9,
 'bacurau': 569,
 'me': 41,
 'senti': 3,
 'ninja': 3,
 'rt': 33,
 'avokdoido': 7,
 'quem': 15,
 'nao': 17,
 'gostou': 9,
 'é': 219,
 'no': 79,
 'mínimo': 13,
 'do': 99,
 'pouco': 9,
 'nazista': 7,
 'sim': 9,
 'vi': 7,
 'ontem': 1,
 'com': 75,
 'a': 213,
 'viviane_cardoso': 1,
 'o': 147,
 'filme': 55,
 'fantástico': 1,
 'vale': 3,
 'pena': 5,
 'ver': 61,
 'se': 57,
 'você': 149,
 'tiver': 1,
 'mente': 1,
 'aberta': 1,
 'e': 213,
 'qualquer': 5,
 'coisa': 9,
 'que': 155,
 'faço': 1,
 'vivi': 1,
 'sempre': 27,
 'bom': 9,
 'companheira': 9,
 'filmes': 3,
 'vida': 45,
 'lutas': 1,
 'lt': 5,
 '3': 7,
 'hoje': 15,
 'finalmente': 3,
 'vejo': 3,
 'será': 1,
 'consigo': 3,
 'chegar': 1,
 'até': 13,
 'sala': 25,
 'livre': 1,
 'tá': 21,
 'difícil': 1,
 'lzanin': 3,
 'texto': 3,
 'marcelo': 5,
 'coelho': 3,
 'chama': 3,
 '‘bolsonarisno': 3,
 'sinal': 3,
 'trocado’': 3,
 'lembra': 3,
 'ideia': 3

# Implementação do Amanda Bynes


In [125]:
prob_irr = {}
prob_rel = {}

for palavra in palavras_totais:
    prob_rel[palavra]= freq_rel[palavra]/(len(palavras_totais)+quantidade_relevancia[1])
    prob_irr[palavra]= freq_irr[palavra]/(len(palavras_totais)+quantidade_relevancia[0])
    
#probabilidade da palavra em relação à relevância do tweet


___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**